In [1]:
import keras
import tensorflow as tf
from keras import layers
from keras.datasets import mnist
import numpy as np
from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
from matplotlib.pyplot import imread

2023-10-04 12:24:16.483096: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 12:24:16.681576: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-04 12:24:16.771456: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-04 12:24:17.503059: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

# **Utils**

In [2]:
def fp_quantize(x, w, f):
    """
    fixed point post quantization.
    Args:
        x: floating point (can be list) 32 bit input
        w: bit width of the target fixed point
        f: fraction bit width of the target fixed point
    Returns:
        the desired fixed point quantized of the input x
    """
    i = w - f
    max = float(2 ** (i - 1) - 2 ** (-f))
    min = float(-2 ** (i - 1))
    n = float(2 ** f)
    xx = np.floor(x * n + 0.5) / n
    clipped = np.clip(xx, a_min=min, a_max=max)
    return clipped

In [3]:
def FPToHex(x, w = 8, f = 7):
    """
    Converts a given Fixed point number to its Hex representation.
    Args:
        x: fixed point having w bits and f bits fraction input
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        Hex representation of input
    """
    x_fix = x
    x_fix = x_fix * pow(2, f)
    x_fix = int(x_fix)
    if(x_fix < 0):
        binary = bin(x_fix+(1<<w))
    else:
        binary = bin(x_fix)
    return hex(int(binary, 2))[2:]

def WriteFixPToFile(file_name, data_fp, w = 8, f = 7, mode = "w"):
    """
    Writes given Fixed point numbers specified by w bits width and f bits fraction to the given file name.
    Args:
        file_name: file name to save the output
        data_fp: fixed point list having w bits and f bits fraction input
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        None
    """
    with open(file_name, mode) as file:
        for i in range(data_fp.shape[0]):
            file.write(str(FPToHex(data_fp[i], w=w, f=f))+'\n')

Use below function if needed (like verifying outputs or debugging).

In [4]:
def twos_complement(bin_num):
    """
    calculates the 2's complement of the given binary number.
    Args:
        bin_num: binary number in string format
    Returns:
        the binary representation after performing 2's complement
    """

    # Perform 2's complement on the binary number
    flipped = ''
    for bit in bin_num:
        flipped += '0' if bit == '1' else '1'
    comp_num = int(flipped, 2) + 1

    return bin(comp_num)[2:].zfill(len(bin_num))

def fixed_point_to_float(bin_num, w=16, f=14):
    """
    Converts a fixed point number given in binary representation to its decimal floating point.
    Args:
        bin_num: binary number in string format
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        decimal floating point
    """
        # Check if the number is negative
    sign_bit = int(bin_num[0])
    if sign_bit:
        bin_num = twos_complement(bin_num, w, f)

    # Split the binary number into integer and fractional parts
    if f > 0:
        if f == w:
            float_num = int(bin_num, 2) / 2**f
        else:
            int_part = int(bin_num[:-f], 2)
            frac_part = int(bin_num[-f:], 2) / 2**f
            float_num = int_part + frac_part
    else:
        int_part = int(bin_num, 2)
        float_num = int_part

    # Apply the sign to the floating point number if it was negative
    if sign_bit:
        float_num = -float_num

    return float_num

def hex_to_fixed_point_decimal(hex_value, w, f):
    """
    Converts a fixed point number given in hex representation to its decimal floating point.
    Args:
        hex_value: hex number in string format
        w: bit width of the input
        f: fraction bit width of the input
    Returns:
        decimal floating point
    """
    binary_value = bin(int(hex_value, 16))[2:]

    # # Determine the sign bit
    binary_value = binary_value.zfill(w)

    # return decimal_value
    return fixed_point_to_float(binary_value, w, f)

# **Preprocess**

In [5]:
path = './notMNIST_large/'

fnames = []
for root, dir, files in os.walk(path):
    for f in files:
        if f.endswith('.png'):
            fnames.append(os.path.join(root, f))

print(len(fnames))
fnames[:3]

264560


['./notMNIST_large/D/QmVsbEdvdGhpYyBCbGsgQlQgQmxhY2sudHRm.png',
 './notMNIST_large/D/QmVhckJ1dHRlVCBJdGFsaWMudHRm.png',
 './notMNIST_large/D/Q2hlbHNleSBJdGFsaWMudHRm.png']

In [6]:
images = []
for f in fnames:
    try:
        img = imread(str(f))
        images.append(img)
    except Exception as e:
        print('Could not read:', f, ':', e, '- it\'s ok, skipping.')

images = np.array(images)

Could not read: ./notMNIST_large/D/VHJhbnNpdCBCb2xkLnR0Zg==.png : not a PNG file - it's ok, skipping.
Could not read: ./notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png : not a PNG file - it's ok, skipping.
Could not read: ./notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png : not a PNG file - it's ok, skipping.
Could not read: ./notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png : not a PNG file - it's ok, skipping.


In [7]:
len(images)

264556

In [11]:
LATENT_SAPCE_DIM = 16
LATENT_SPACE_WIDTH = 4
LATENT_SAPCE_HEIGHT = 4
IMAGE_SIZE = 28

In [8]:
x_train, x_test = train_test_split(images, test_size=0.2, random_state=42)

In [9]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

In [10]:
def resize_mnist_images(images, new_size=(28, 28)):
    """
    resizes a list of images.
    Args:
        images: list of images
        new_size: the new desired size
    Returns:
        list of resized images
    """
    resized_images = []
    for img in images:
        resized_img = cv2.resize(img, new_size)
        resized_images.append(resized_img)
    return np.array(resized_images)